In [191]:
import pandas as pd
import os, sys 

https://medium.com/@sandyshah1990/langchain-agents-and-function-calling-using-llama-2-locally-29ce057e4789

### Libraries

In [192]:
from langchain_ollama.llms import OllamaLLM
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document

from langchain_community.embeddings.ollama import OllamaEmbeddings

from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

In [193]:
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS

### Load  requirements

In [195]:
# Carregar o arquivo Excel
file_path = "Template - Requisito de Software v02.xlsx"
xls = pd.ExcelFile(file_path)

# Carregar abas específicas
sobre_documento = pd.read_excel(xls, sheet_name="Sobre o documento")
glossario = pd.read_excel(xls, sheet_name="1-Glossário")
requisitos = pd.read_excel(xls, sheet_name='2-Requisitos de Produto', skiprows= len(xls.sheet_names)+1)
lists = pd.read_excel(xls, sheet_name="3-Lists")
versoes = pd.read_excel(xls, sheet_name="4-Versões")
guia = pd.read_excel(xls, sheet_name="5-Guia")

requisitos= requisitos.dropna(subset=['Requisito','Categoria'])
# Exibir as primeiras linhas da aba de requisitos
#print(requisitos.head())

requisitos_texto = requisitos.to_string(index=False)

/opt/conda/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [5]:
requisitos.head()

,Req ID,Categoria,Descrição Alto Nível,Requisito,Comentários,Status,Histórico,Versão de modificação,Origem,Jira issue,Test Case Jira
0,PRS-00001,Requisitos Funcionais,RF - Modos de Operação,O dispositivo deve ser capaz de operar em cinc...,NaN,Novo,NaN,v_1,NaN,NaN,NaN
1,PRS-00002,Requisitos Funcionais,RF - Modos de Operação,O dispositivo deve permitir a ativação do modo...,configurado pelo Json,Novo,NaN,v_1,NaN,NaN,NaN
2,PRS-00003,Requisitos Funcionais,RF - Modos de Operação,O dispositivo deve permitir a ativação do modo...,configurado pelo Json,Novo,NaN,v_1,NaN,NaN,NaN
3,PRS-00004,Requisitos Funcionais,RF - Modos de Operação,O dispositivo deve permitir a ativação do modo...,configurado pelo Json,Novo,NaN,v_1,NaN,NaN,NaN
4,PRS-00005,Requisitos Funcionais,RF - Modos de Operação,O dispositivo deve permitir a ativação do modo...,configurado pelo Json,Novo,NaN,v_1,NaN,NaN,NaN


### Format requirements in docs

In [6]:
from langchain.docstore.document import Document

# Criar lista de documentos
docs = []
for _, row in requisitos.iterrows():
    content = f"ID: {row['Req ID']} | Categoria: {row['Categoria']} | Descrição Alto Nível: {row['Descrição Alto Nível']} | Requisito: {row['Requisito']}"
    docs.append(Document(page_content=content, metadata={"ID": row["Req ID"]}))

In [7]:
import json
docs_list =[]  
for _, row in requisitos.iterrows():
    doc = {
        "id": row['Req ID'],
        "requisito": row['Requisito']
    }
    docs_list.append(doc)

# Convertendo para JSON
docs_json = json.dumps(docs_list, indent=4, ensure_ascii=False)

### LLM

In [165]:
from langchain.llms import Ollama

llm_mistral = Ollama(model="mistral:latest", temperature=0.1)

llm_llama31 = Ollama(model="llama3.1:8b", temperature=0.1)

llm_deepseek_7b = Ollama(model="deepseek-r1:7b", temperature=1)
llm_deepseek_8b = Ollama(model="deepseek-r1:8b", temperature=1)
llm_deepseek_14b = Ollama(model="deepseek-r1:14b", temperature=1)

llm_llama3 = Ollama(model="llama3.3:latest", temperature=0.1)
llm_deepseek_70b = Ollama(model="deepseek-r1:70b", temperature=1)
#llm_reqs_indiv = Ollama(model="deepseek-r1:1.5b", temperature=0)

###  Available all requirement 

In [147]:
from langchain.prompts import PromptTemplate
import json
docs_list = []
def analisar_todos(docs_json, llm_reqs_glob):
    prompt = """
        Avalie cada requisito com base nas boas práticas descritas abaixo:
        
        Boas Práticas para Requisitos:
        1. Os requisitos devem ser claros, precisos e não genéricos.
        2. Os requisitos devem ter sentenças curtas.
        3. Os requisitos devem ser individuais.
        4. Os requisitos devem ser independentes.
        5. Os requisitos devem ser consistentes.
        6. Os requisitos devem ser completos.
        7. Os requisitos devem ser verificáveis.
        8. Os requisitos não devem ser repetidos ou redundantes.

        Requisitos:
        {requisitos}


        Sua resposta deve ser assim para cada requisito:
        ID:
        Requisito:
        Avaliação:
        Clareza:
        Sentenças Curtas:
        Individualidade:
        Independência:
        Consistência:
        Completude:
        Verificabilidade:
        Repetição:
        Classificação (classificar em requisito de Produto ou Sistema):
        """

    prompt_template = PromptTemplate(
        input_variables=["requisitos"], template=prompt)

    prompt  = prompt_template.format(requisitos=docs_json)
    analise = llm_reqs_glob(prompt)
    return analise

In [148]:
analise_global = analisar_todos(docs_json, llm_deepseek_70b)
print(analise_global)

<think>
Okay, so I'm trying to figure out how to evaluate each of these requirements. There are a lot of them, but I guess I'll start with the first one and see how it goes.

First up is ID: PRS-00001. The requirement says the device must have an API for integration with third-party systems. That sounds pretty clear to me. But wait, what exactly does "third-party systems" mean here? Are we talking about any specific types of systems or just any external software? Also, which communication protocols are supported? It doesn't specify HTTP, REST, WebSocket, etc. So maybe the requirement isn't entirely complete because it lacks some details.

Clareza: I think this is clear in stating that an API is needed, but without specifics on protocols, it's a bit vague. Maybe a 7/10 for clareza?

Sentenças Curtas: It’s concise, so that's good. No issues here.

Individualidade: Yeah, it stands alone as a requirement about integration via API.

Independência: This doesn't depend on other requirements; 

In [149]:
analise_global = analisar_todos(docs_json, llm_llama3)
print(analise_global)

Vou avaliar cada requisito de acordo com os critérios fornecidos. Aqui está a avaliação do primeiro requisito:

ID: PRS-0001
Requisito: O dispositivo deve realizar a amplificação de sinal, incorporando um amplificador de baixo ruído que deve fornecer um ganho ajustável na faixa de 30 dB a 40dB
Avaliação: Este requisito é bem definido e específico.
Clareza: Alta - O requisito é claro e fácil de entender.
Sentenças Curtas: Sim - O requisito é conciso e direto.
Individualidade: Sim - O requisito trata de uma funcionalidade específica do dispositivo.
Independência: Sim - O requisito não depende de outros requisitos para ser entendido.
Consistência: Alta - O requisito é consistente com os demais requisitos relacionados à funcionalidade do dispositivo.
Completude: Alta - O requisito fornece todos os detalhes necessários para sua implementação.
Verificabilidade: Sim - É possível verificar se o dispositivo atende a este requisito.
Repetição: Não - Este requisito não é repetido em outros itens 

###  Global relations analysis

In [141]:
from langchain.prompts import PromptTemplate
import json
docs_list = []
def analisar_relacoes(docs_json, llm_reqs_glob):
    prompt = """
        Analise os seguintes requisitos como um todo:
        {requisitos}

        Identifique:
        1. Redundâncias: Requisitos que se sobrepõem ou repetem informações.
        2. Dependências: Requisitos que dependem de outros para serem compreendidos ou implementados.
        3. Inconsistências: Requisitos que se contradizem ou usam terminologia inconsistente.

        Forneça uma análise detalhada e obrigatoriamente responda em português.
        """

    prompt_template = PromptTemplate(
        input_variables=["requisitos"], template=prompt)

    prompt  = prompt_template.format(requisitos=docs_json)
    analise = llm_reqs_glob(prompt)
    return analise

In [142]:
analise_global = analisar_relacoes(docs_json, llm_deepseek_70b)
print(analise_global)

<think>
Okay, so I need to analyze a list of requirements provided for a device and identify redundancies, dependencies, and inconsistencies. Let me start by going through each requirement one by one to understand what they're asking for.

First, looking at the list, I notice that several requirements are about physical buttons. For example, there's PRS-00323 which mentions a power button, PRS-00324 and 325 talk about UP and DOWN buttons, and PRS-00327 refers to quick access buttons without specifying what they're for. This repetition might indicate redundancy because multiple buttons are mentioned without clear differentiation.

Next, some requirements mention specific dimensions and weight (PRS-00329, 30, 31). These seem related because knowing the device's size and weight is important for design, but maybe there’s overlap in how they’re presented. I should check if any of these could be combined or are unnecessarily duplicated.

There are also several requirements about transducers:

In [118]:
analise_global = analisar_relacoes(docs_json, llm_llama3)
print(analise_global)

Após analisar a lista de requisitos fornecida, identifiquei as seguintes questões:

**Redundâncias:**

1. Requisitos PRS-00329 e PRS-00331 se referem às dimensões do dispositivo e da placa, respectivamente. Embora não sejam idênticos, podem ser considerados redundantes, pois ambos tratam de dimensões físicas.
2. Requisitos PRS-00334, PRS-00335 e PRS-00336 se referem à frequência de amostragem, canais físicos para transmissão e recepção, respectivamente. Embora não sejam idênticos, podem ser considerados redundantes, pois todos tratam de aspectos relacionados à capacidade de processamento do dispositivo.
3. Requisitos PRS-00342 e PRS-00343 se referem à bateria interna recarregável e autonomia mínima de uso contínuo, respectivamente. Embora não sejam idênticos, podem ser considerados redundantes, pois ambos tratam de aspectos relacionados à bateria do dispositivo.

**Dependências:**

1. Requisito PRS-00340 (identificar o transdutor conectado) depende do requisito PRS-00339 (operar com os

### Individual Analysis

In [167]:
def avaliar_requisito(requisito, llm_reqs_indiv):
    prompt = """
    Avalie o seguinte requisito com base nas boas práticas descritas abaixo, responda em português:
    
    Boas Práticas para Requisitos:
    1. Os requisitos devem ser claros, precisos e não genéricos.
    2. Os requisitos devem ter sentenças curtas.
    3. Os requisitos devem ser individuais.
    4. Os requisitos devem ser independentes.
    5. Os requisitos devem ser consistentes.
    6. Os requisitos devem ser completos.
    7. Os requisitos devem ser verificáveis.
    8. Os requisitos não devem ser repetidos ou redundantes.
    9. Os requisitos devem ser coerentes quanto ao conteúdo.
    10. Os requisitos não devem repetir várias vezes a mesma palavra.
    
    Exemplo 1:
    Requisito: "O sistema deve permitir que o usuário faça login usando seu e-mail e senha."
    
    Avaliação:
    Clareza: Sim, o requisito é claro e específico.
    Sentenças Curtas: Sim, a sentença é curta e direta.
    Individualidade: Sim, trata-se de um único requisito.
    Independência: Sim, este requisito não depende de outro.
    Consistência: Sim, o requisito é consistente com as boas práticas.
    Completude: Sim, o requisito descreve completamente a funcionalidade.
    Verificabilidade: Sim, é possível verificar se o login funciona com e-mail e senha.
    Repetição: Não há repetição desnecessária de palavras.
    
    Testável: Sim. Teste: Verificar se o usuário consegue fazer login inserindo e-mail e senha válidos.
    Classificação: Sistema (é uma funcionalidade do sistema).
    
    Exemplo 2:
    Requisito: "O sistema deve ser rápido."
    
    Avaliação:
    Clareza: Não, o requisito é vago e genérico. O que significa "rápido"?
    Sentenças Curtas: Sim, a sentença é curta.
    Individualidade: Sim, trata-se de um único requisito.
    Independência: Sim, este requisito não depende de outro.
    Consistência: Não, o requisito é inconsistente porque "rápido" não é definido.
    Completude: Não, o requisito não descreve completamente a funcionalidade.
    Verificabilidade: Não, não é possível verificar se o sistema é "rápido" sem uma definição clara.
    Repetição: Não há repetição desnecessária de palavras.
    
    Testável: Não. Não há como testar algo tão subjetivo.
    Classificação: Sistema (é uma característica geral do sistema).
    
    Exemplo 3:
    Requisito: "O sistema deve enviar uma notificação ao usuário quando houver uma nova mensagem."
    
    Avaliação:
    Clareza: Sim, o requisito é claro e específico.
    Sentenças Curtas: Sim, a sentença é curta e direta.
    Individualidade: Sim, trata-se de um único requisito.
    Independência: Sim, este requisito não depende de outro.
    Consistência: Sim, o requisito é consistente com as boas práticas.
    Completude: Sim, o requisito descreve completamente a funcionalidade.
    Verificabilidade: Sim, é possível verificar se a notificação é enviada.
    Repetição: Não há repetição desnecessária de palavras.
    
    Testável: Sim. Teste: Verificar se o usuário recebe uma notificação ao receber uma nova mensagem.
    Classificação: Sistema (é uma funcionalidade do sistema).
    
    Exemplo 4:
    Requisito: "O sistema deve ser fácil de usar."
    
    Avaliação:
    Clareza: Não, o requisito é vago e subjetivo. O que significa "fácil de usar"?
    Sentenças Curtas: Sim, a sentença é curta.
    Individualidade: Sim, trata-se de um único requisito.
    Independência: Sim, este requisito não depende de outro.
    Consistência: Não, o requisito é inconsistente porque "fácil de usar" não é definido.
    Completude: Não, o requisito não descreve completamente a funcionalidade.
    Verificabilidade: Não, não é possível verificar algo tão subjetivo.
    Repetição: Não há repetição desnecessária de palavras.
    
    Testável: Não. Não há como testar algo tão subjetivo.
    Classificação: Produto (é uma característica geral do produto).
    
    Agora, avalie o seguinte requisito:
    
    Requisito: {requisito}
    
    Sua resposta deve seguir o mesmo formato acima:
    
    Avaliação:
    Clareza: 
    Sentenças Curtas: 
    Individualidade: 
    Independência: 
    Consistência: 
    Completude: 
    Verificabilidade: 
    Repetição: 
    
    Testável (Identifique se o Requisito é Testável, se sim indique o teste):
    """
    prompt_template = PromptTemplate(
    input_variables=["requisito"], template=prompt)

    prompt  = prompt_template.format(requisito=requisito)
    analise = llm_reqs_indiv(prompt)
    return analise

In [168]:
requisitos['Requisito'][0]

'O dispositivo deve ser capaz de operar em cinco modos de operação distintos, que incluam: Modo B, Modo M, Modo Doppler colorido e Doppler power, PW Doppler'

In [169]:
req = requisitos['Requisito'][0]
analise = avaliar_requisito(req, llm_deepseek_70b)
print(analise[analise.find('\n</think>\n')+9:])



Avaliação do Requisito:

- **Clareza**: Sim, o requisito é claro ao listar modos específicos de operação.

- **Sentenças Curtas**: Parcialmente, a sentença é um pouco longa devido à lista, mas ainda assim compreensível.

- **Individualidade**: Não, pois agrupa vários modos em um único requisito; seria melhor dividir em requisitos separados.

- **Independência**: Sim, cada modo funciona independentemente.

- **Consistência**: Sim, todos os modos são consistentes entre si.

- **Completude**: Sim, lista explicitamente os cinco modos.

- **Verificabilidade**: Sim, cada modo pode ser testado individualmente.

- **Repetição**: Não há repetições desnecessárias.

**Testável**: Sim. Teste: Verificar se o dispositivo opera corretamente em cada um dos modos listados (Modo B, Modo M, Modo Doppler colorido, Doppler power, PW Doppler).

**Classificação**: Produto (relaciona-se à funcionalidade do dispositivo).


In [170]:
req = requisitos['Requisito'][0]
analise = avaliar_requisito(req, llm_deepseek_14b)
print(analise[analise.find('\n</think>\n')+9:])



Avaliação:
Clareza: Sim, o requisito é claro e específico ao listar todos os cinco modos de operação necessários.  
Sentenças Curtas: Não, a sentença é um pouco longa e pode ser dividida para maior clareza.  
Individualidade: Sim, trata-se de um único requisito.  
Independência: Sim, este requisito não depende de outro.  
Consistência: Sim, o requisito é consistente com as boas práticas descritas.  
Completude: Sim, o requisito descreve completamente a funcionalidade necessária.  
Verificabilidade: Sim, é possível verificar se o dispositivo opera corretamente em cada um dos modos listados.  
Repetição: Não há repetição desnecessária de palavras.  

Testável (Identifique se o Requisito é Testável, se sim indique o teste):  
Sim. Teste: Verificar se o dispositivo consegue operar corretamente em cada um dos modos listados (Modo B, Modo M, Modo Doppler colorido, Doppler power e PW Doppler).  

Classificação (classifique se é um requisito de Produto ou Sistema):  
Sistema.


In [171]:
req = requisitos['Requisito'][0]
analise = avaliar_requisito(req, llm_mistral)
print(analise)

 Avaliação:
   Clarez: Sim, o requisito é claro e específico.
   Sentenças Curtas: Sim, a sentença é curta e direta.
   Individualidade: Sim, trata-se de um único requisito.
   Independência: Sim, este requisito não depende de outro.
   Consistência: Sim, o requisito é consistente com as boas práticas.
   Completude: Sim, o requisito descreve completamente a funcionalidade.
   Verificabilidade: Sim, é possível verificar se o dispositivo opera em cinco modos específicos.
   Repetição: Não há repetição desnecessária de palavras.

   Testável (Identifique se o Requisito é Testável, se sim indique o teste): Sim. Teste: Verificar se o dispositivo opera em cinco modos específicos: Modo B, Modo M, Modo Doppler colorido e Doppler power, PW Doppler.
   Classificação (classifique se é um requisito de Produto ou Sistema): Produto (é uma funcionalidade do dispositivo).


In [172]:
req = requisitos['Requisito'][0]
analise = avaliar_requisito(req, llm_llama3)

print(f"Requisito: {req}")
print(analise)

Requisito: O dispositivo deve ser capaz de operar em cinco modos de operação distintos, que incluam: Modo B, Modo M, Modo Doppler colorido e Doppler power, PW Doppler
Avaliação:
Clareza: Sim, o requisito é claro e específico, pois lista os modos de operação distintos que o dispositivo deve ser capaz de operar.
Sentenças Curtas: Não, a sentença é um pouco longa devido à enumeração dos modos de operação.
Individualidade: Sim, trata-se de um único requisito, embora especifique várias funcionalidades relacionadas.
Independência: Sim, este requisito não depende explicitamente de outro para sua compreensão ou implementação.
Consistência: Sim, o requisito é consistente com as boas práticas, pois especifica claramente os modos de operação necessários.
Completude: Sim, o requisito descreve completamente a funcionalidade requerida do dispositivo em termos dos modos de operação.
Verificabilidade: Sim, é possível verificar se o dispositivo pode operar nos modos especificados através de testes adeq

In [173]:
req = requisitos['Requisito'][0]
analise = avaliar_requisito(req, llm_llama31)

print(f"Requisito: {req}")
print(analise)

Requisito: O dispositivo deve ser capaz de operar em cinco modos de operação distintos, que incluam: Modo B, Modo M, Modo Doppler colorido e Doppler power, PW Doppler
Avaliação:

Clareza: Sim, o requisito é claro e específico ao descrever os cinco modos de operação distintos do dispositivo.

Sentenças Curtas: Sim, a sentença é curta e direta, listando os modos de operação sem ambiguidades.

Individualidade: Sim, trata-se de um único requisito que descreve uma funcionalidade específica do dispositivo.

Independência: Sim, este requisito não depende de outro para ser compreendido ou implementado.

Consistência: Sim, o requisito é consistente com as boas práticas ao descrever claramente os modos de operação.

Completude: Sim, o requisito descreve completamente a funcionalidade do dispositivo em relação aos modos de operação.

Verificabilidade: Sim, é possível verificar se o dispositivo opera corretamente nos cinco modos de operação listados.

Repetição: Não há repetição desnecessária de p

'O dispositivo deve possuir indicador visual do nível da bateria na IHM (interface do usuário - física)'

In [174]:
from tqdm import tqdm

resultados_individual = []

for req in tqdm(json.loads(docs_json)):
    analise = avaliar_requisito(req['requisito'], llm_llama31)
    resultados_individual.append(analise)

# Adicionar resultados ao DataFrame
requisitos['Análise Individual'] = resultados_individual

100%|██████████| 47/47 [01:57<00:00,  2.51s/it]


In [176]:
resultados_individual[0]

'Avaliação:\n\nClareza: Sim, o requisito é claro e específico. Ele descreve exatamente quais são os modos de operação que o dispositivo deve ser capaz de realizar.\n\nSentenças Curtas: Sim, a sentença é curta e direta, listando os modos de operação distintos que o dispositivo deve suportar.\n\nIndividualidade: Sim, trata-se de um único requisito, que descreve uma funcionalidade específica do dispositivo.\n\nIndependência: Sim, este requisito não depende de outro, pois descreve uma característica autônoma do dispositivo.\n\nConsistência: Sim, o requisito é consistente com as boas práticas, pois é claro e específico sobre os modos de operação que o dispositivo deve suportar.\n\nCompletude: Sim, o requisito descreve completamente a funcionalidade, listando todos os modos de operação distintos que o dispositivo deve ser capaz de realizar.\n\nVerificabilidade: Sim, é possível verificar se o dispositivo opera corretamente em cada um dos cinco modos de operação mencionados.\n\nRepetição: Não 

In [ ]:
from tqdm import tqdm

resultados_individual = []

for req in tqdm(json.loads(docs_json)):
    analise = avaliar_requisito2(str(req))
    resultados_individual.append(analise[analise.find('\n</think>\n')+9:])

# Adicionar resultados ao DataFrame
requisitos['Análise Individual'] = resultados_individual

100%|██████████| 47/47 [08:24<00:00, 10.74s/it]


In [ ]:
json.loads(docs_json)[0]

{'id': 'PRS-00001',
 'requisito': 'O dispositivo deve ser capaz de operar em cinco modos de operação distintos, que incluam: Modo B, Modo M, Modo Doppler colorido e Doppler power, PW Doppler'}

In [184]:
with open('analise_completa.txt', 'w') as f:
    # Análise Individual
    f.write("### Análise Individual:\n")
    for i, analise in enumerate(resultados_individual):
        Id = json.loads(docs_json)[i]['id']
        requisito =  json.loads(docs_json)[i]['requisito']
        f.write(f"ID: {id} | Requisito {requisito}: \n{analise}\n\n")

    # Análise Global
    f.write("### Análise Global:\n")
    f.write(analise_global)